In [0]:
%sql

/*CREATE CATALOG RX_DRUGS
MANAGED LOCATION 'abfss://rx-drugs-processed@rxdrugs.dfs.core.windows.net';
CREATE SCHEMA RX_DRUGS.RxDrugs
GRANT USAGE ON SCHEMA RX_DRUGS.RxDrugs TO `andrewh@arhsvcs.com`
*/



In [0]:
from pyspark.sql import SparkSession

container = "rx-drugs-processed"
storage_Account = "rxdrugs"
file_path =f'abfss://{container}@{storage_Account}.dfs.core.windows.net/RX-Drugs.csv'

df = spark.read.csv(file_path,header=True, inferSchema=True)
df.display()
##wasbs://rx-drugs-processed@rxdrugs.blob.core.windows.net

In [0]:
from pyspark.sql.functions import col, sum, when
from datetime import datetime

DataQualityFlag = True

##checking for nulls in columns
print("Table below shows the number of nulls in each column")
null_counts = df.select(
    [sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
)
DataQualityFlag = False if null_counts.count() > 0 else True
null_counts.display()

##Now check for none intergers in the TotalCost column
non_integer_totalcost = df.filter(~col("TotalCost").cast("int").isNotNull())
if non_integer_totalcost.count() > 0:
    print("There are non-integer values in the TotalCost column")
    non_integer_totalcost.select("TotalCost").display()
    DataQualityFlag = False
else:
    print("Checked - All entries in the TotalCost are integers")
##Write to Databricks catalog if Data Quality Check fails
df.write.mode("overwrite").saveAsTable(
    "RX_Drugs.RxDrugs.RejRXTable"
) if DataQualityFlag == False else print("Data Quality Check Passed")

#Replace NULLS if the name of the column stating that no data entered for that record.
replacement_Values = {col: f'{col} is not stated for this record' for col in df.columns}
df_filled = df.fillna(replacement_Values)
df_filled.display()

#Save cleansed table to the catalog.
current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime("%Y%m%d_%H%M%S")
table_name = f"RX_DRUGS.RxDrugs.cleansedRXTable_{formatted_datetime}"

df_filled.write.mode("overwrite").saveAsTable(table_name)

